In [ ]:
import pandas as pd
from google.colab import files

files.upload()
!ls

df = pd.read_excel('reg_experience.xlsx', index_col=0)#Data Frame
df

Saving reg_experience.xlsx to reg_experience.xlsx
reg_experience.xlsx  sample_data


,x,y,t,z,u
id,,,,,
0,132,188,105,15,1834
1,88,225,76,22,2045
2,89,183,119,15,1653
3,79,259,48,17,2270
4,117,160,138,13,1555
...,...,...,...,...,...
195,67,224,83,14,1967
196,112,144,160,26,1373
197,108,228,77,16,2083


Построим модель с предикторами $x, y, z.$

В этой модели все будет хорошо. Эти предикторы некоррелированы.

Построим модель с предикторами $x, y, z, t.$

В этой модели тоже все будет неплохо, хотя предикторы t и y коррелированы.


In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(df[['x', 'y', 'z']], df['u'], test_size=0.5, random_state = 1)

reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

print(reg.coef_)
y_pred = reg.predict(X_test)
print(r2_score(y_test, y_pred))

[1.96696314 8.03223047 0.82563986]
0.991607325061953


В датасет добавим два "предиктора", которые по сути являются шумом. Это $a$ и $b$.

Еще добавим предиктор $c$, являющийся линейной комбинацией предикторов $x$ и $y$.

И вот теперь-то с определением коэффициентом по МНК все станет плохо.

Вопрос.
1. Что является причиной плохой оценки параметров модели? Причина в "предикторах" $a$ и $b$ или в $c$?
2. Как влияет амплитуда шума noiseAmplitude на качество оценки параметров? Почему?


In [ ]:
import numpy as np

sampleSize = len(df) # 200 observations
noiseAmplitude = 0.5
tmp = 20 * np.random.randn(sampleSize, 2)
tmp = tmp.round(0)
df[['a', 'b']] = pd.DataFrame(tmp)
df['c'] = df['x'] + df['y'] #+ noiseAmplitude * np.random.randn(sampleSize)
df

,x,y,t,z,u,a,b,c
id,,,,,,,,
0,132,188,105,15,1834,41.0,28.0,320
1,88,225,76,22,2045,10.0,21.0,313
2,89,183,119,15,1653,-33.0,-34.0,272
3,79,259,48,17,2270,-10.0,15.0,338
4,117,160,138,13,1555,6.0,15.0,277
...,...,...,...,...,...,...,...,...
195,67,224,83,14,1967,20.0,8.0,291
196,112,144,160,26,1373,-31.0,9.0,256
197,108,228,77,16,2083,18.0,-21.0,336


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
import numpy as np

columns =['x', 'y', 'z', 't', 'a', 'b', 'c']
#columns = ['x', 'y', 'z', 't', 'c']
res = []
for i in range(40):
  X_train, X_test, y_train, y_test = train_test_split(df[columns], df['u'], test_size=0.5, random_state=i)

  reg = linear_model.LinearRegression()
  #reg = linear_model.Ridge(alpha=10)
  #reg = linear_model.Lasso(alpha=20)
  reg.fit(X_train, y_train)

  y_pred = reg.predict(X_test)
  #print(r2_score(y_test, y_pred))
  
  row = reg.coef_.round(4).tolist()
  row.append(reg.intercept_.round(4).tolist())
  res.append(row)

res = pd.DataFrame(res)
res.describe()